# Finetune DeepSeek for Legal Consultation

## Credits

This notebook and this code is a fork of Abid Ali Awan's tutorial on [Fine-Tuning DeepSeek R1](https://www.datacamp.com/tutorial/fine-tuning-deepseek-r1-reasoning-model). Eternally grateful for his work with DataCamp and his contributions to the community.


## Which tools & packages will we be using today?

Packages we're going to be using throughout this walkthrough will be

- `unsloth`: Efficient fine-tuning and inference for LLMs — Specifically we will be using:
    - `FastLanguageModel` module to optimize inference & fine-tuning
    - `get_peft_model` to enable LoRa (Low-Rank Adaptation) fine-tuning
- `peft`: Supports LoRA-based fine-tuning for large models.
- Different Hugging Face modules:
    - `transformers` from HuggingFace to work with our fine-tuning data and handle different model tasks
    - `trl` Transformer Reinforcement Learning from HuggingFace which allows for supervised fine-tuning of the model — we will use the `SFFTrainer` wrapper
    - `datasets` to fetch reasoning datasets from the Hugging Face Hub
- `torch`: Deep learning framework used for training
- `wandb`: Provides access to weights and biases for tracking our fine-tuning experiment 

## Before we get started — how to access the Hugging Face and Weights & Biases API

### Set GPU accelerator
We are using Kaggle Notebooks because we have access to free GPUs. To enable GPU access, press on Settings > Accelerator > GPU T4 x2

### How to access the Hugging Face API

1. Register to Huggin Face if you have not already
2. Go to [Hugging Face Tokens](https://huggingface.co/settings/tokens).
3. Click **"New Token"**.
4. Select **read/write** permissions if needed.
5. Copy your **API key**.

### Weights & Biases API key**
1. Sign up at [Weights & Biases](https://wandb.ai/site).
2. Go to [W&B Settings](https://wandb.ai/settings).
3. Copy your **API key** from the "API Keys" section.

### Add the API keys to Kaggle Notebooks
1. Press on Add-ons > Secrets
2. Add the API keys under `Hugging_Face_Token` and `wnb` respectively

You can now use this code to retrieve your API keys

```py
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hugging_face_token = user_secrets.get_secret("Hugging_Face_Token")
wnb_token = user_secrets.get_secret("wnb")
```

## Install relevant packages

In [3]:
%%capture
!pip install unsloth # install unsloth

## Import all relevant packages throughout this walkthrough

In [4]:
import re
import html

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from trl import SFTTrainer

from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset, Dataset

import wandb
from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Create API keys and login to Hugging Face and Weights and Biases

In [5]:
# Initialize Hugging Face & WnB tokens
user_secrets = UserSecretsClient() # from kaggle_secrets import UserSecretsClient
hugging_face_token = user_secrets.get_secret("Hugging_Face_Token")
wnb_token = user_secrets.get_secret("wnb")

# Login to Hugging Face
login(hugging_face_token) # from huggingface_hub import login

# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project="DeepSeek-legaladvice-finetune", 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: juh6973 (juh6973-tampere-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Loading DeepSeek R1 and the Tokenizer

**What are we doing in this step?**

In this step, we **load the DeepSeek R1 model and its tokenizer** using `FastLanguageModel.from_pretrained()`. We also **configure key parameters** for efficient inference and fine-tuning. We will be using a distilled 8B version of R1 for faster computation.  

**Key parameters explained**
```py
max_seq_length = 2048  # Define the maximum sequence length a model can handle (i.e., number of tokens per input)
dtype = None  # Default data type (usually auto-detected)
load_in_4bit = True  # Enables 4-bit quantization – a memory-saving optimization
```

**Intuition behind 4-bit quantization**

Imagine compressing a **high-resolution image** to a smaller size—**it takes up less space but still looks good enough**. Similarly, **4-bit quantization reduces the precision of model weights**, making the model **smaller and faster while keeping most of its accuracy**. Instead of storing precise **32-bit or 16-bit numbers**, we compress them into **4-bit values**. This allows **large language models to run efficiently on consumer GPUs** without needing massive amounts of memory. 

In [6]:
# Set parameters
max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default 
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization 

# Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # Load the pre-trained DeepSeek R1 model (8B parameter version)
    max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
    dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
    load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
    token=hugging_face_token, # Use hugging face token
)

==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Declare EOS token

In [7]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<｜end▁of▁sentence｜>'

## Testing DeepSeek R1 on a legal use-case before fine-tuning


### Defining a system prompt 
To create a prompt style for the model, we will define a system prompt and include placeholders for the question and response generation. The prompt will guide the model to think step-by-step and provide a logical, accurate response.

In [8]:
# Define a system prompt under prompt_style 
PROMPT_STYLE = """Below is an instruction describing a legal task, accompanied by additional context. 
Write a comprehensive response that appropriately addresses the request. 
Before responding, methodically think through the legal issue step-by-step to ensure a logical, accurate, and persuasive answer.

### Instruction:
You are a legal expert with advanced knowledge in legal reasoning, case analysis, statutory interpretation, and legal argumentation. 
Please respond to the following legal inquiry.

### Legal Inquiry:
{}

### Response:
{}"""

### Running inference on the model

In this step, we **test the DeepSeek R1 model** by providing a **legal inquiry** and generating a response.  
The process involves the following steps:

1. **Define a test question** related to a legal case.
2. **Format the question using the structured prompt (`prompt_style`)** to ensure the model follows a logical reasoning process.
3. **Tokenize the input and move it to the GPU (`cuda`)** for faster inference.
4. **Generate a response using the model**, specifying key parameters like `max_new_tokens=1200` (limits response length).
5. **Decode the output tokens back into text** to obtain the final readable answer.

In [9]:
# Creating a legal test question for inference
question = """A small business owner signs a contract with a supplier to deliver goods by a specific date. 
            The supplier fails to deliver the goods on time, causing significant financial losses to the business owner. 
            The contract does not explicitly specify damages for late delivery. Considering contract law principles, 
            what legal remedies could the business owner seek, and how would a court likely assess the claim for damages in this scenario?
            """

# Enable optimized inference mode for Unsloth models (improves speed and efficiency)
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# Format the question using the structured prompt (`prompt_style`) and tokenize it
inputs = tokenizer([PROMPT_STYLE.format(question, "")], return_tensors="pt").to("cuda")  # Convert input to PyTorch tensor & move to GPU

# Generate a response using the model
outputs = model.generate(
    input_ids=inputs.input_ids, # Tokenized input question
    attention_mask=inputs.attention_mask, # Attention mask to handle padding
    max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
    use_cache=True, # Enable caching for faster inference
)

# Decode the generated output tokens into human-readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the relevant response part (after "### Response:")
print(response[0].split("### Response:")[1])  


The business owner has several potential legal remedies available under contract law principles due to the supplier's failure to deliver goods on time, despite the contract not explicitly specifying damages for late delivery.

First, the business owner can claim lost profits or actual damages under the doctrine of reliance, which holds the supplier accountable for the losses directly resulting from their breach. Courts typically calculate these damages by considering the contract's terms, such as the agreed-upon price and the value of the goods, to determine the extent of the loss.

Second, the business owner may seek specific performance, but this remedy is less likely to be granted if the goods are no longer available or if the delay has caused significant inconvenience. However, if the supplier can still fulfill the contract, specific performance might be an appropriate remedy.

Third, the business owner could request a breach of contract damages, which are intended to put the non-

>**Before starting fine-tuning — why are we fine-tuning in the first place?**
>
> Even without fine-tuning, our model successfully generated a chain of thought and provided reasoning before delivering the final answer. The reasoning process is encapsulated within the `<think>` `</think>` tags. So, why do we still need fine-tuning? The reason is taht we want the final answer to be consistent in a certain style (legal arguments). 



### Step 2 — Download the fine-tuning dataset and format it for fine-tuning

We'll use the Pile of Law dataset to conduct the fine-tuning. It contains legal advice from subreddit [r/legaladvice](https://www.reddit.com/r/legaladvice/) and is a good fine-tune for concise question answering. The dataset itself can be found from [HuggingFace](https://huggingface.co/datasets/pile-of-law/pile-of-law)

In [10]:
# Download the dataset using Hugging Face — function imported using from datasets import load_dataset
n_sample = 10000
dataset = load_dataset(
    "pile-of-law/pile-of-law",
    "r_legaladvice",
    split="train",
    streaming=True,
    trust_remote_code=True,
).take(n_sample)
dataset

README.md:   0%|          | 0.00/25.6k [00:00<?, ?B/s]

pile-of-law.py:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

The repository for pile-of-law/pile-of-law contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/pile-of-law/pile-of-law.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60


IterableDataset({
    features: ['text', 'created_timestamp', 'downloaded_timestamp', 'url'],
    num_shards: 1
})

Function to clean HTML tags and special characters

In [11]:
def clean_text(text):
    text = html.unescape(text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'&[^;\s]+;', '', text)
    
    return text.strip()

Function to extract structured data

In [12]:
def extract_fields(text):
    text = clean_text(text)

    title_match = re.search(r'Title:\s*(.*?)\s*Question:', text, re.DOTALL)
    title = title_match.group(1).strip() if title_match else ""

    question_match = re.search(r'Question:\s*(.*?)(Answer #1:)', text, re.DOTALL)
    question = question_match.group(1).strip() if question_match else ""

    answers_matches = re.findall(r'Answer #\d+:\s*(.*?)(?=Answer #\d+:|$)', text, re.DOTALL)
    combined_answers = "\n\n".join(ans.strip() for ans in answers_matches)

    # Combine Title and Question
    full_question = f"{title}\n\n{question}" if title else question

    return full_question, combined_answers

Function to format prompts clearly

In [13]:
def formatting_prompts_func(examples):
    inputs = []
    outputs = []

    for text in examples["text"]:
        question, combined_answers = extract_fields(text)
        if question and combined_answers:
            inputs.append(question)
            outputs.append(combined_answers)

    texts = []
    for input, output in zip(inputs, outputs):
        formatted_text = PROMPT_STYLE.format(input, output) + EOS_TOKEN
        texts.append(formatted_text)

    return {"text": texts}

Format the dataset

In [14]:
formatted_dataset = formatting_prompts_func({"text": [item["text"] for item in dataset]})
formatted_dataset = Dataset.from_dict(formatted_dataset)

Error reading file: https://huggingface.co/datasets/pile-of-law/pile-of-law/resolve/main/data/train.r_legaldvice.jsonl.xz


Inspect the first formatted sample

In [15]:
print(formatted_dataset)
print(formatted_dataset[0]["text"])

Dataset({
    features: ['text'],
    num_rows: 9997
})
Below is an instruction describing a legal task, accompanied by additional context. 
Write a comprehensive response that appropriately addresses the request. 
Before responding, methodically think through the legal issue step-by-step to ensure a logical, accurate, and persuasive answer.

### Instruction:
You are a legal expert with advanced knowledge in legal reasoning, case analysis, statutory interpretation, and legal argumentation. 
Please respond to the following legal inquiry.

### Legal Inquiry:
Landlord broke lease agreement, what are my rights? (Chicago, IL)

Our landlord has been promising us a washer/dryer unit since we moved in (July 2015). When we resigned the lease August 2016, we wrote into the lease that an in-unit washer and dryer would be installed by September 30th 2016.

Since September 30th, there have been continuous delays in getting the W/D installed. Since it has now been almost a month past the date the W/

>**Next step is to structure the fine-tuning dataset according to train prompt style—why?**
>
> - Each question is paired with chain-of-thought reasoning and the final response.
> - Ensures every training example follows a consistent pattern.
> - Prevents the model from continuing beyond the expected response lengt by adding the EOS token.

### Step 3 — Setting up the model using LoRA

**An intuitive explanation of LoRA** 

Large language models (LLMs) have **millions or even billions of weights** that determine how they process and generate text. When fine-tuning a model, we usually update all these weights, which **requires massive computational resources and memory**.

LoRA (**Low-Rank Adaptation**) allows to fine-tune efficiently by:

- Instead of modifying all weights, **LoRA adds small, trainable adapters** to specific layers.  
- These adapters **capture task-specific knowledge** while leaving the original model unchanged.  
- This reduces the number of trainable parameters **by more than 90%**, making fine-tuning **faster and more memory-efficient**.  

Think of an LLM as a **complex factory**. Instead of rebuilding the entire factory to produce a new product, LoRA **adds small, specialized tools** to existing machines. This allows the factory to adapt quickly **without disrupting its core structure**.

For a more technical explanation, check out this tutorial by [Sebastian Raschka](https://www.youtube.com/watch?v=rgmJep4Sb4&t).

Below, we will use the `get_peft_model()` function which stands for Parameter-Efficient Fine-Tuning — this function wraps the base model (`model`) with LoRA modifications, ensuring that only specific parameters are trained.

In [16]:
# Apply LoRA (Low-Rank Adaptation) fine-tuning to the model 
FastLanguageModel.for_training(model)
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank: Determines the size of the trainable adapters (higher = more parameters, lower = more efficiency)
    target_modules=[  # List of transformer layers where LoRA adapters will be applied
        "q_proj",   # Query projection in the self-attention mechanism
        "k_proj",   # Key projection in the self-attention mechanism
        "v_proj",   # Value projection in the self-attention mechanism
        "o_proj",   # Output projection from the attention layer
        "gate_proj",  # Used in feed-forward layers (MLP)
        "up_proj",    # Part of the transformer’s feed-forward network (FFN)
        "down_proj",  # Another part of the transformer’s FFN
    ],
    lora_alpha=16,  # Scaling factor for LoRA updates (higher values allow more influence from LoRA layers)
    lora_dropout=0,  # Dropout rate for LoRA layers (0 means no dropout, full retention of information)
    bias="none",  # Specifies whether LoRA layers should learn bias terms (setting to "none" saves memory)
    use_gradient_checkpointing="unsloth",  # Saves memory by recomputing activations instead of storing them (recommended for long-context fine-tuning)
    random_state=3407,  # Sets a seed for reproducibility, ensuring the same fine-tuning behavior across runs
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (disabled here, meaning fixed-rank LoRA is used)
    loftq_config=None,  # Low-bit Fine-Tuning Quantization (LoFTQ) is disabled in this configuration
)

Unsloth 2025.3.17 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Now, we initialize `SFTTrainer`, a supervised fine-tuning trainer from `trl` (Transformer Reinforcement Learning), to fine-tune our model efficiently on a dataset.

In [17]:
def is_bfloat16_supported(): 
    return torch.cuda.is_bf16_supported() if torch.cuda.is_available() else False

In [18]:
# Initialize the fine-tuning trainer — Imported using from trl import SFTTrainer
trainer = SFTTrainer(
    model=model_lora,  # The model to be fine-tuned
    tokenizer=tokenizer,  # Tokenizer to process text inputs
    train_dataset=formatted_dataset,  # Dataset used for training
    dataset_text_field="text",  # Specifies which field in the dataset contains training text
    max_seq_length=max_seq_length,  # Defines the maximum sequence length for inputs
    dataset_num_proc=2,  # Uses 2 CPU threads to speed up data preprocessing

    # Define training arguments
    args = TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples processed per device (GPU) at a time
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps before updating weights
        num_train_epochs=1,  # Full fine-tuning run
        warmup_steps=5,  # Gradually increases learning rate for the first 5 steps
        #max_steps=60,  # Limits training to 60 steps (useful for debugging; increase for full fine-tuning)
        learning_rate=2e-4,  # Learning rate for weight updates (tuned for LoRA fine-tuning)
        fp16=True,  # Use FP16 (if BF16 is not supported) to speed up training
        bf16=False,  # Use BF16 if supported (better numerical stability on newer GPUs)
        logging_steps=10,  # Logs training progress every 10 steps
        optim="adamw_8bit",  # Uses memory-efficient AdamW optimizer in 8-bit mode
        weight_decay=0.01,  # Regularization to prevent overfitting
        lr_scheduler_type="linear",  # Uses a linear learning rate schedule
        seed=420,  # Sets a fixed seed for reproducibility
        output_dir="outputs",  # Directory where fine-tuned model checkpoints will be saved
    ),
)


Spawning 2 processes


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/9997 [00:00<?, ? examples/s]

Concatenating 2 shards


## Step 4 — Model training

The training results can be checked on Weights and Biases

In [ ]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,997 | Num Epochs = 1 | Total steps = 1,249
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.940400
20,2.303600
30,2.153000
40,2.182400
50,2.158600
60,2.192700
70,2.134400
80,2.051300
90,2.104300
100,2.110600


Save the fine-tuned model

In [ ]:
wandb.finish()

## Save the model to Hugging Face

In [ ]:
new_model_online = "Juh6973/DeepSeek-R1-Legal-Advice-COT"
model_lora.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

The model can be found at https://huggingface.co/Juh6973/DeepSeek-R1-Legal-Advice-COT